In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id='notebook_id'></a>
# Overriding `equals`: Part 1

The `equals` method is one of the more important methods in Java. Every object reference can be used to invoke `equals` because `equals` is a non-final `public` method of the class `java.lang.Object`. Many classes have methods that rely on a correctly implemented `equals` method; for example, the `List` method `contains` invokes `equals` on the elements of list until an element matching the searched for object is found.

This notebook is concerned with classes that are not part of an inheritance hierarchy. A notebook in Part 3 of the notes discusses issues that arise when overriding `equals` in classes that are part of an inheritance hierarchy.

## When to override `equals`

The simplest solution to the problem of overriding `equals` is to simply not override `equals`. If a class does not override `equals` then it inherits the `equals` implementation from `Object` which implements equality of *identity* (an object is only equal to itself).

A class should override `equals` when objects of the class are compared for *logical* equality. Logical equality means equality of state. Most classes that represent some kind of value should probably implement logical equality. Some examples include:

* the `String` class overrides `equals` so that two `String`s are equal if and only if they represent the same sequence of characters
* the `ArrayList` class overrides `equals` so that two `ArrayList`s are equal if and only if they represent the same sequence of elements
* a class that represents the time of day should probably override `equals` so that two times are equal if and only if they represent the same time of the day
* a class that represents a two-dimensional point should probably override `equals` so that two points are equal if and only if they have equal coordinates

There are many classes where the notion of logical equality is meaningless or not useful. 

Utility classes should not override `equals` because utility classes are not used to create objects of the utility class type. For example, the `Math` class does not override `equals`.

Some classes represent values that are unique; if such a class ensures that there is only one instance for each unique value then there is no need to override `equals` because a unique instance can only be equal to itself. Some examples include:

* an enumeration is a special kind of class where there are fixed number of unique instances whose states never change; see the [Enumerations](enumerations.ipynb#notebook_id) notebook for a discussion of the enumerations
* Canadian Social Insurance Numbers (SINs) are supposed to be unique
* student numbers are supposed to be unique
* serial numbers are supposed to be unique


## The `equals` contract

Many classes, including the collections classes in the Java Collections Framework, depend on objects having an `equals` method that obeys the contract detailed in the [`equals` documentation](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/Object.html#equals(java.lang.Object)). The `equals` contract has five parts. For non-null references an implementation of `equals` must support the following requirements:

1. reflexive
2. symmetric
3. transitive
4. consistent
5. non-nullity

For the mathematically inclined, the `equals` contract describes an [equivalence relation](https://en.wikipedia.org/wiki/Equivalence_relation). 

### Reflexivity

*Reflexivity*  means that an object is equal to itself. In other words, for a non-null reference `x` the value of `x.equals(x)` must be `true`.


### Symmetry

*Symmetry* means that two objects must agree on whether they are equal. In other words, for non-null references `x` and `y` the value of `x.equals(y)` is equal to the value of `y.equals(x)`.

### Transitivity

*Transitiviy* means that if first object is equal to a second object and the second object is equal to a third object then the first object must be equal to the third object. In other words, for non-null references `x`, `y`, and `z`, if

```
x.equals(y)
```

is equal to `true` and if

```
y.equals(z)
```

is equal to `true` then

```
x.equals(z)
```

must also be equal to `true`.

### Consistency

*Consistency* means that the value returned by `equals` must not change if the states of the compared objects do not change.

### Non-nullity

The non-nullity requirement means that for a non-null reference `x` the value of `x.equals(null)` is always `false`; furthermore, `x.equals(null)` never throws an exception.

## Recipe for ensuring the `equals` contract

Joshua Bloch published a step-by-step recipe for implementing a high-quality `equals` method in the book *Effective Java, Third Edition*. The recipe for overriding `equals` where `equals` has the header:

```java
public boolean equals(Object obj)
```

is as follows:

1. Return `true` if `this == obj` is equal to `true`. This ensures that the reflexivity requirement is satisfied. Technically this step is not necessary because the remaining steps will also ensure that reflexivity is satisfied.
2. Use `instanceof` to check if `obj` has the correct type and return `false` if `obj` does not have the correct type. In the absence of inheritance, the correct type is the class in which `equals` is being implemented.
3. Cast `obj` to the correct type. Again, in the absence of inheritance, the correct type is the class in which `equals` is being implemented.
4. For every field that matters for determining if two instances of the class are equal, test if the field belonging to `this` object is equal to the field belonging to the argument object. If all of these tests succeed then return `true` and otherwise return `false`.
    1. for primitive fields that are not `float` or `double` use `==` to compare the fields for equality
    2. for `float` fields use the static method [`Float.compare(float, float)`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/Float.html#compare(float,float)) to compare the fields for equality
    3. for `double` fields use the static method [`Double.compare(double, double)`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/Double.html#compare(double,double)) to compare the fields for equality
    4. for reference fields use the `equals` method to compare the fields for equality; for array fields consider using one of the `Arrays.equals` methods
        1. if the class allows a reference field to have the value `null` then use the static method [`Objects.equals(Object, Object)`](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/util/Objects.html#equals(java.lang.Object,java.lang.Object)) to safely compare the fields for equality



## Overriding `Counter.equals(Object)`

We will walk through an example of implementing `equals` for the `Counter` class while discussing each step of the recipe in greater depth. For the `Counter` class we will say that two `Counter` instances are equal if and only if their current values are equal. Furthermore, a `Counter` can be equal only to another `Counter` (and not to, for example, an `Integer` or a `String`).

To begin, we add the `equals` method to the class and assume that the return value is `false`.

```java
public class Counter {

    private int value;

    public Counter(int value) {
        if (value < 0) {
            throw new IllegalArgumentException("value must be non-negative");
        }
        this.value = value;
    }
    
    // other constructors and methods not shown

    @Override
    public boolean equals(Object obj) {
        
        return false;
    }
}
```

### `Counter.equals(Object)` Step 1

Step 1 of the recipe says to test if `this == obj` and to return `true` if the test returns `true`. This requires a simple `if` statement:

In [ ]:
public class Counter {

    private int value;

    public Counter(int value) {
        if (value < 0) {
            throw new IllegalArgumentException("value must be non-negative");
        }
        this.value = value;
    }
    
    // other constructors and methods not shown

    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        return false;
    }
}

Recall that `this` is a reference to the object that was used to invoke `equals` and that the `==` operator checks if two references refer to the same object; thus, the `if` statement is simply testing that `this` and `obj` are the same object and if they are the same object then they must be equal.

After completing Step 1, the `Counter` `equals` method satisfies the reflexivity requirement of the `equals` contract:

In [ ]:
Counter c = new Counter(0);
Counter d = c;
System.out.println(c.equals(d));

Of course it fails for two different `Counter`s having the same value, but we will address this in Step 4.

In [ ]:
Counter c = new Counter(0);
Counter d = new Counter(0);
System.out.println(c.equals(d));

### Counter.equals(Object) Step 2

Step 2 of the recipe says to use `instanceof` to check if `obj` has the correct type and return `false` if `obj` does not have the correct type.

The [`instanceof` operator](https://docs.oracle.com/javase/tutorial/java/nutsandbolts/op2.html) compares an object reference to a specified type. The expression

```java
x instanceof SomeClass
```

where `x` is a reference variable evaluates to `true` if the type of `x` is the same as `SomeClass` (or if the type of `x` is a subclass of `SomeClass` which is discussed in the Inheritance notebooks). If `x` is `null` then the expression evaluates to `false`.

Using the `instanceof` operator to implement Step 2 of the recipe looks something like:

In [ ]:
public class Counter {

    private int value;

    public Counter(int value) {
        if (value < 0) {
            throw new IllegalArgumentException("value must be non-negative");
        }
        this.value = value;
    }
    
    // other constructors and methods not shown

    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        // Step 2
        if (!(obj instanceof Counter)) {
            return false;
        }
        
        return false;
    }
}

The `if` statement in Step 2 tests that obj is *not* a `Counter` reference. If the test passes then `obj` cannot be equal to `this` `Counter` because `obj` does not refer to a `Counter`. If `obj` is `null` then the `instanceof` statement evaluates to `false` and the `if` statement body returns `false` without throwing an exception which satisfies the non-nullity requirement of the `equals` contract.

### Counter.equals(Object) Step 3

If execution of the `equals` method continues past Step 2 then we know that `obj` is in fact a `Counter` object. To complete testing for equality we need to access the `value` field of the `Counter` but we cannot do so yet because the parameter `obj` is declared to be `Object` and `Object` instances do not have fields named `value`.

Step 3 of the recipe says to cast `obj` to the correct type which in this case is the type `Counter`. The cast is required before the compiler will allow us to use the `value` field of `obj`. Implementing the cast required by Step 3 looks lik:

In [ ]:
public class Counter {

    private int value;

    public Counter(int value) {
        if (value < 0) {
            throw new IllegalArgumentException("value must be non-negative");
        }
        this.value = value;
    }
    
    // other constructors and methods not shown

    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        // Step 2
        if (!(obj instanceof Counter)) {
            return false;
        }
        
        // Step 3
        // Casting obj to a Counter is required because in the parameter
        // list of the method obj is declared to be an Object and Object
        // instances have no fields (that we know of)
        Counter other = (Counter) obj;
        
        return false;
    }
}

After Step 3 we have a non-null reference `other` that we can use to get the value of the `Counter` that `obj` refers to.

### Counter.equals(Object) Step 4

Step 4 of the equals recipe says to test that every field of `this` `Counter` is equal to the corresponding field of the `other` `Counter`. If all of these tests succeed then return `true` and otherwise return `false`.

A `Counter` only has one field so Step 4 requires only a simple `if` statement:

In [ ]:
public class Counter {

    private int value;

    public Counter(int value) {
        if (value < 0) {
            throw new IllegalArgumentException("value must be non-negative");
        }
        this.value = value;
    }
    
    // other constructors and methods not shown

    
    /**
     * Compares this counter to the specified object. The result is {@code true} if
     * and only if the argument is not {@code null} and is a {@code Counter} object
     * that has the same current value as this object.
     * 
     * @param obj the object to compare this counter against
     * @return true if the given object represents a Counter with the same current
     *         value as this counter, false otherwise
     */
    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        // Step 2
        if (!(obj instanceof Counter)) {
            return false;
        }
        
        // Step 3
        Counter other = (Counter) obj;
        
        // Step 4
        // After Step 3, we can access the value of the counter using
        // the variable other.
        if (this.value == other.value) {
            return true;
        }
        return false;
    }
}

Step 4 tests if the value of `this` `Counter` is equal to the value of the `other` `Counter` returning `true` if the test passes. If the two counters have different values then they are not equal and the method returns `false`.

Note that the field `value` has type `int` and the recipe says to use `==` to compare `this.value` and `other.value` for equality. See the `Point2.equals(Object)` implementation below for an example where the fields have type `double`.

In [ ]:
Counter c = new Counter(10);
System.out.println(c.equals(null));

It also returns `false` when we try to compare a `Counter` to some other type such as a `String`:

In [ ]:
Counter c = new Counter(0);
String s = "not a counter";
System.out.println(c.equals(s));

## Overriding `Point2.equals(Object)`



When comparing `float` or `double` values in an `equals` method you should prefer using the static methods `Float.compare` and `Double.compare` to perform the comparison instead of `==` because of the existence of the values `Float.NaN`, `Double.NaN`, `-0.0f`, and `-0.0`. The `compare` methods allow `NaN` values to be equal (unlike `==` where `NaN` is never equal to any value). The `compare` methods consider floating-point `-0` to be not equal to `+0` (unlike `==` where `-0f == 0f` and `-0.0 == 0.0` are both equal to `true`). The explanation of why this matters can be found in the [Overriding `hashCode`](./overriding_hashcode.ipynb#notebook_id) notebook.

`Double.compare(val1, val2)` returns `0` if `val1` and `val2` are equal subject to the exceptions described in the previous paragraph; see the [documentation here](https://docs.oracle.com/en/java/javase/11/docs/api/java.base/java/lang/Double.html#compare(double,double)).

Steps 1-3 of implementing `equals` always results in very similar code. In Step 4, we need to compare the $x$ and $y$ coordinates of the two points for equality. Following the recipe, we use `Double.compare` to perform the comparision. An implementation of `equals` for a minimal version of the `Point2` class is shown in the following cell:

In [ ]:
public class Point2 {
    
    private double x;
    private double y;

    public Point2(double x, double y) {
        this.x = x;
        this.y = y;
    }

    // other constructors and methods not shown
    
    /**
     * Compares this point to the specified object. The result is {@code true} if
     * and only if the argument is not {@code null} and is a {@code Point2} object
     * that has the same x and y coordinates as this object.
     * 
     * @param obj the object to compare this point against
     * @return true if the given object represents a Point2 with the same x and y
     *         coordinates as this point, false otherwise
     */
    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        // Step 2
        if (!(obj instanceof Point2)) {
            return false;
        }
        
        // Step 3
        Point2 other = (Point2) obj;
        
        // Step 4
        // use Double.compare instead of == to compare double fields
        // use Float.compare instead of == to compare float fields
        if (Double.compare(this.x, other.x) == 0 && 
                Double.compare(this.y, other.y) == 0) {
            return true;
        }
        return false;
    }

## Overriding `Card.equals(Object)`

The `Card` class has two reference type fields (both of type `String`). A card is equal to another card if and only if the two cards have equal ranks and suits. Because the fields are of reference type, we compare them using the `String.equals(Object)` method in Step 4.

An implementation of `equals` for a minimal version of the `Card` class is shown in the following cell:

In [ ]:
import java.util.Arrays;

public class Card {
    private String rank;
    private String suit;
    
    public static final String[] RANKS = {
        "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"
    };
    
    public static final String[] SUITS = {
        "CLUBS", "DIAMONDS", "HEARTS", "SPADES"
    };
    
    /**
     * Initializes this card to have the specified rank and suit.
     *
     * @param rank the rank of this card
     * @param suit the suit of this card
     */
    public Card(String rank, String suit) {
        if (!Arrays.asList(RANKS).contains(rank)) {
            throw new IllegalArgumentException();
        }
        if (!Arrays.asList(SUITS).contains(suit)) {
            throw new IllegalArgumentException();
        }
        this.rank = rank;
        this.suit = suit;
    }
    
    // other methods not shown
    
    /**
     * Compares this card to the specified object for equality. The
     * result is true if {@code obj} is a {@code Card} having the same
     * rank and the same suit as this card, false otherwise.
     * 
     * @return true if obj is a card having the same
     * rank and the same suit as this card, false otherwise
     */
    @Override
    public boolean equals(Object obj) {
        // Step 1
        if (this == obj) {
            return true;
        }
        
        // Step 2
        if (!(obj instanceof Card)) {
            return false;
        }
        
        // Step 3
        Card other = (Card) obj;
        
        // Step 4
        // use fieldName.equals to compare reference-type fields
        if (this.rank.equals(other.rank) && this.suit.equals(other.suit)) {
            return true;
        }
        return false;
    }
}

## Exercises

1. Implement the `equals` method for the `Domino` class shown below.

In [ ]:
public class Domino {

    /**
     * The smallest possible value for a side of a domino.
     */
    public static final int MIN_VALUE = 0;
    
    /**
     * The largest possible value for a side of a domino. 
     */
    public static final int MAX_VALUE = 6;

    private int val1;
    private int val2;

    public Domino(int value1, int value2) {
        if (!isValueOK(value1) || !isValueOK(value2)) {
            throw new IllegalArgumentException();
        }
        this.val1 = value1;
        this.val2 = value2;
    }

    public static boolean isValueOK(int value) {
        return value >= MIN_VALUE && value <= MAX_VALUE;
    }
    
    public int getSmallerValue() {
        int result = this.val1 <= this.val2 ? this.val1 : this.val2;
        return result;
    }

    public int getLargerValue() {
        int result = this.val1 >= this.val2 ? this.val1 : this.val2;
        return result;
    }
    
    /**
     * Compares this domino to the specified object. The result is
     * true if and only if the argument is a {@code Domino} object having the
     * same values as this domino object. Note that the side on which
     * a value appears is not considered for the purposes of equals;
     * for instance, {@code eq} will be true in the example below:
     * 
     * <pre>
     * Domino a = new Domino(1, 4);
     * Domino b = new Domino(4, 1);
     * boolean eq = a.equals(b);
     * </pre>
     * 
     * @param obj an object to compare
     * @return true if this domino is equal to the specified object,
     *         and false otherwise
     */
    @Override
    public boolean equals(Object obj) {
        
    }
}


2. See Exercises 3-6 in the [Designing simple classes](./designing_simple_classes.ipynb#notebook_id) notebook. Implement suitable `equals` methods for these classes.

3. See Exercise 5 in the [Constructors](./constructors.ipynb#notebook_id) notebook. Implement `equals` for your complex number class.

4. See Exercise 9 in the [Constructors](./constructors.ipynb#notebook_id) notebook. Implement `equals` for your card game hand class.

5. A `CombinationLock` represents a lock that is unlocked with a sequence of $4$ digits between 0 and 9. Two combination locks are equal if they have the same combination. Implement `equals` for the `CombinationLock` class.

In [ ]:
import java.util.List;
import java.util.ArrayList;

public class CombinationLock {
    
    private List<Integer> combo;
    
    public CombinationLock(List<Integer> combo) {
        if (combo.size() != 4) {
            throw new IllegalArgumentException("combination requires 4 digits");
        }
        this.combo = new ArrayList<>(combo);
    }
    
    @Override
    public String toString() {
        return this.combo.toString();
    }
    
    @Override
    public boolean equals(Object obj) {
        
    }
}

6. Suppose that a `CombinationLock` can have any number of digits. How does the `equals` method implementation change?